In [216]:
from instagrapi import Client
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm
import re

load_dotenv()

True

In [113]:
# get username and password from .env file
USERNAME = os.getenv("ACCOUNT_USERNAME")
PASSWORD = os.getenv("ACCOUNT_PASSWORD")

In [114]:
# login to instagram
cl = Client()
cl.login(USERNAME, PASSWORD)

True

Behavioral data: This includes user interactions, such as likes, comments, shares, and follows, which can reveal interests, preferences, and social networks.

# Scrape Users Following

In [141]:

# init read users from csv
# insert all users you want to scrape data in the users.csv file
users_list = pd.read_csv("data/users.csv")["users"].tolist()

In [184]:
# get all users following data
users_following = []

for user in tqdm(users_list):
    following_list = []
    # followers_list = []
    
    user_id = cl.user_id_from_username(user)
    user_info = cl.user_info(user_id)
    follower_count = user_info.follower_count
    following = cl.user_following(user_id)
    
    for key, val in following.items():
        following_list.append(val.username)
        
    following = re.sub(r'[\[\]\']', '', str(following_list))    
    
    users_following.append({
        'userId': user_id,
        'username':user,
        'following_count': len(following_list),
        'followers_count': follower_count,
        'following_list': following,
        # # 'followers_list': followers_list
                                })

In [185]:
df_following = pd.DataFrame(users_following)
df_following

,userId,username,following_count,followers_count,following_list
0,1067259270,google,34,13733489,"googlepixel_us, googleworkspace, googleplay, y..."
1,2248835396,sugab.3gp,920,543,"jennierubyjane, redvelvet.smtown, chaeyo.0, it..."
2,2286735082,ranggasaputra4000,297,164,"marvelstudios, detikcom, nasro_768, adriantrip..."
3,44378210506,googleindonesia,10,123697,"googleuk, googleartsculture, googlearabia, wom..."
4,8536113897,googleindia,17,923570,"twinklerkhanna, vickykaushal09, googleworkspac..."
5,7140177628,jennierubyjane,0,74728090,
6,3514890218,marvelstudios,88,38438210,"zendaya, marvellatam, gingerthejester, jameela..."
7,1443380468,twinklerkhanna,139,7106115,"whowhatwear, murugaofficial, mindful_madre, th..."
8,4775555891,googlearabia,15,145542,"unhcr_arabic, unicef_mena, unwomenarabic, susa..."


In [218]:
df_following.to_csv("data/users_following.csv", index=False)

---

# Scrape User Comments on Post
## Comments on His Post

In [206]:
target_user = "magangmerdeka"
medias = cl.user_medias(cl.user_id_from_username(target_user), amount=5)

In [225]:
target_user_comment = []

for media in tqdm(medias):
    print("Media ID: ", media.pk)
    media_comments = cl.media_comments(media.pk, amount=500)
    
    for comment in tqdm(media_comments):
        is_user = False
        
        if comment.user.username == target_user:
            is_user = True
        target_user_comment.append({
            'date_time': comment.created_at_utc,
            'is_user_post': is_user,
            'user_id': comment.user.pk,
            'username': comment.user.username,
            'has_like': comment.has_liked,
            'like_count': comment.like_count,
            'comment_id': comment.pk,
            'comment': comment.text,
                                    })

  0%|          | 0/5 [00:00<?, ?it/s]

Media ID:  3040703176500040778


 20%|██        | 1/5 [00:03<00:12,  3.18s/it]

Media ID:  3040628760243908723


0it [00:00, ?it/s]
 40%|████      | 2/5 [00:04<00:06,  2.15s/it]

...

 60%|██████    | 3/5 [00:30<00:26, 13.16s/it]

Media ID:  3037207643546326339


 80%|████████  | 4/5 [01:08<00:22, 22.99s/it]

Media ID:  3037097323897268103


100%|██████████| 5/5 [01:16<00:00, 15.33s/it]


In [226]:
df_target_user_comment = pd.DataFrame(target_user_comment)
df_target_user_comment

,date_time,is_user_post,user_id,username,has_like,like_count,comment_id,comment
0,2023-02-20 22:35:58+00:00,False,5911210509,pipiterika_,False,0.0,17969587667010469,Dapat uang saku gaksih
1,2023-02-20 11:15:33+00:00,False,7567114522,0tiekz_,None,NaN,18206124454214345,@rwrt1.0 kelas full english kak batch ini
2,2023-02-20 07:17:14+00:00,False,40533453762,novemberboy__,False,0.0,18081435949333130,"Saya, saya bang ga lolos msib"
3,2023-02-20 02:37:45+00:00,False,54031801044,belajaradmin,False,0.0,17976118007045613,🔥🔥
4,2023-02-20 01:46:22+00:00,False,4268345080,fajaroktaladi,None,NaN,17954490824437563,@syanasblaa lanjut dm kak
...,...,...,...,...,...,...,...,...
575,2023-02-13 07:16:58+00:00,False,1817674405,ryan.als,None,NaN,18234761251089516,@spontanahuy tanggal 15 terakhirnya 🙏🏻 ada bbr...
576,2023-02-13 07:15:30+00:00,False,11559681202,dzakimf25,None,NaN,17985681952871212,@spontanahuy ga ush berharap lgi udh tgl segin...
577,2023-02-13 07:05:39+00:00,False,4014876406,zulfah__amalia1607,None,NaN,17924041316568555,@spontanahuy asliii butuh kepastian
578,2023-02-13 07:05:32+00:00,False,7779084511,lindamanullang07,False,1.0,18266225029191291,Siapa saja yg lolos utk ikut survei diinfokan ...


In [227]:
df_target_user_comment.to_csv("data/target_user_comment.csv", index=False)